In [1]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    GenerationConfig,  
    DynamicCache, StaticCache)
import accelerate
import torch
import csv
import time
import os
import json
import inspect

In [2]:
from transformers import logging

In [3]:
#logging.disable_progress_bar()

In [4]:
access_tk = os.getenv("HF_API_TOKEN")
device_str = "cuda:0"
#device_str = "auto"

cache_dir = os.getenv("HF_HOME", os.path.expanduser("~/.cache/huggingface/hub/"))
print(cache_dir)

#model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-3.2-3B-Instruct" #"meta-llama/Llama-3.2-1B-Instruct"
#model_name = "EleutherAI/gpt-neo-1.3B"

padding_side='left'

model = AutoModelForCausalLM.from_pretrained(model_name, token=access_tk, device_map=device_str, 
                                             torch_dtype=torch.float16, cache_dir=cache_dir)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_tk, 
                                          torch_dtype=torch.float16, padding_side=padding_side, 
                                          cache_dir=cache_dir)

/home/daniel/.cache/huggingface/hub/


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [5]:
model.config.__dict__.keys()

dict_keys(['vocab_size', 'max_position_embeddings', 'hidden_size', 'intermediate_size', 'num_hidden_layers', 'num_attention_heads', 'num_key_value_heads', 'hidden_act', 'initializer_range', 'rms_norm_eps', 'pretraining_tp', 'use_cache', 'rope_theta', 'rope_scaling', 'attention_bias', 'attention_dropout', 'mlp_bias', 'head_dim', 'return_dict', 'output_hidden_states', 'output_attentions', 'torchscript', 'torch_dtype', 'use_bfloat16', 'tf_legacy_loss', 'pruned_heads', 'tie_word_embeddings', 'chunk_size_feed_forward', 'is_encoder_decoder', 'is_decoder', 'cross_attention_hidden_size', 'add_cross_attention', 'tie_encoder_decoder', 'max_length', 'min_length', 'do_sample', 'early_stopping', 'num_beams', 'num_beam_groups', 'diversity_penalty', 'temperature', 'top_k', 'top_p', 'typical_p', 'repetition_penalty', 'length_penalty', 'no_repeat_ngram_size', 'encoder_no_repeat_ngram_size', 'bad_words_ids', 'num_return_sequences', 'output_scores', 'return_dict_in_generate', 'forced_bos_token_id', 'forc

In [6]:
# max_new_tokens = model.config.max_position_embeddings - 2048
max_new_tokens = 1500

In [7]:
# prompt_cache = DynamicCache()
# prompt_cache.get_max_cache_shape()
#prompt_cache = StaticCache(config=model.config, max_batch_size=10, max_cache_len=max_new_tokens, 
#                           device="cuda:0", dtype=torch.bfloat16)

In [8]:
model.device

device(type='cuda', index=0)

In [9]:
type(model.config.name_or_path)

str

In [10]:
model.config.is_encoder_decoder

False

In [11]:
model.num_parameters()/1e6

3212.749824

In [12]:
model.config.num_hidden_layers

28

In [13]:
model.config.hidden_size

3072

In [14]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [15]:
[elem/1e9 for elem in torch.cuda.mem_get_info()]

[17.953193984, 25.425608704]

In [16]:
[elem/1e9 for elem in torch.cuda.mem_get_info()]

[17.953193984, 25.425608704]

In [17]:
tokenizer.model_max_length

131072

In [18]:
model.config.max_position_embeddings

131072

In [19]:
tokenizer.pad_token

In [20]:
tokenizer.pad_token_id

In [21]:
tokenizer.pad_token = tokenizer.eos_token
add_generation_prompt = True
padding = True
return_tensors = "pt"

In [22]:
tokenizer.pad_token

'<|eot_id|>'

In [23]:
type(tokenizer.pad_token_id)

int

In [17]:
generation_config = GenerationConfig.from_pretrained(model_name, token=access_tk)

generation_config.output_hidden_states = True
generation_config.output_logits = True
generation_config.return_dict_in_generate = True
generation_config.max_new_tokens = max_new_tokens
generation_config.pad_token_id = tokenizer.pad_token_id
generation_config.use_cache = True

In [23]:
print(f"LLAMA GenerationConfig. \n\n \
Generation Strategy: \n \
\t- do_sample: {generation_config.do_sample} \n \
\t- num_beams: {generation_config.num_beams} \n\n \
Model Output Logit manipulation: \n \
\t- top-k: {generation_config.top_k} \n \
\t- top-p: {generation_config.top_p} \n \
\t- min-p: {generation_config.min_p} \n \
\t- temperature: {generation_config.temperature}\n")

LLAMA GenerationConfig. 

 Generation Strategy: 
 	- do_sample: True 
 	- num_beams: 1 

 Model Output Logit manipulation: 
 	- top-k: 29 
 	- top-p: 0.9 
 	- min-p: None 
 	- temperature: 0.6



In [32]:
generation_config.top_k = 123

In [33]:
model.config.top_k = 0.24

In [34]:
model.config.top_k

0.24

In [75]:
generation_config.use_cache

True

In [76]:
#print(*generation_config.__dict__.keys(), sep="\n")

In [77]:
# sentences = ["What is serendipity? Explain in maximum one paragraph."]
# sentences = [
#     {"role": "macaroni", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "What is serendipity? Explain in maximum one paragraph."}
# ]

sentences=[{'role': 'system', 'content': "You are an expert in combinatorial optimization over graphs, and I need you to solve a problem"},
            {'role': "user", "content": "Hi ChatGPT. Do you think you can tell me the sum of a and b?"}]

print(tokenizer.decode(tokenizer.apply_chat_template(sentences, 
                                                     padding=padding, 
                                                     return_tensors=return_tensors, 
                                                     add_generation_prompt=add_generation_prompt)[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 11 Apr 2025

You are an expert in combinatorial optimization over graphs, and I need you to solve a problem<|eot_id|><|start_header_id|>user<|end_header_id|>

Hi ChatGPT. Do you think you can tell me the sum of a and b?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [78]:
sentences = [[{"role": "user", 
               "content": "Can you extend the given Python code to include more abbreviations and their full names that you might be familiar with?"\
               " Also, can you modify the code to print the abbreviations and their full names in alphabetical order?"}]]*10

In [79]:
sentences=[[{'role': 'user', 
             'content': "When did Virgin Australia start operating? Answer in 3 sentences.\nContext: Virgin Australia, the trading name of " \
             "Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin "\
             "brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as"\
             "a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown"\
             "to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."}]]*10

In [80]:
inputs = tokenizer.apply_chat_template(sentences, padding=padding, return_tensors='pt', 
                                       add_generation_prompt=add_generation_prompt,
                                       return_attention_mask=True)

In [81]:
inputs.shape

torch.Size([10, 155])

In [82]:
# Manually create attention mask (1s for non-padding tokens)
attention_mask = torch.ones_like(inputs).to(device_str)

In [83]:
output = model.generate(inputs.to(device_str), attention_mask=attention_mask, generation_config=generation_config)

In [91]:
type(output)

transformers.generation.utils.GenerateDecoderOnlyOutput

In [84]:
output.keys()

odict_keys(['sequences', 'logits', 'hidden_states', 'past_key_values'])

In [85]:
output['sequences'] = output.sequences.detach().cpu()

In [86]:
output.sequences.device

device(type='cpu')

In [87]:
type(output.hidden_states[0][0])

torch.Tensor

In [88]:
output.__dict__.keys()

dict_keys(['sequences', 'scores', 'logits', 'attentions', 'hidden_states', 'past_key_values'])

In [89]:
type(output.logits[0])

torch.Tensor

In [90]:
output.logits[0].shape

torch.Size([10, 128256])

In [40]:
def get_len_out_tokens_batch(input_size, output_seq, tokenizer):
    assert type(output_seq) == torch.Tensor, "ERROR: output must a tensor representing the batch of token outputs"
    tmp_output = output_seq[:,input_size:]
    # Create a tensor with same size as output filled with the pad_token_id
    eos_mask = torch.ones(tmp_output.shape, dtype=tmp_output.dtype).to(tmp_output.device)*tokenizer.pad_token_id
    # Return sum of all values of output along batch axis
    return torch.sum(tmp_output != eos_mask, dim=1) + 1

In [41]:
out_lengths = get_len_out_tokens_batch(inputs.shape[1], output['sequences'], tokenizer)
print(f"Length of the outputs (in # of tokens):\n{out_lengths}")

Length of the outputs (in # of tokens):
tensor([67, 66, 77, 58, 77, 67, 60, 83, 76, 85])


In [42]:
output_str = tokenizer.batch_decode(output['sequences'][:, inputs.shape[-1]:], skip_special_tokens=True)

In [43]:
for i,elem in enumerate(output_str):
    print(f"ELEMENT {i}: \n{elem}")
    print(f"SIZE OF ELEM: {out_lengths[i]}")
    print("============================================================\n")

ELEMENT 0: 
Virgin Australia, trading as Virgin Australia Airlines Pty Ltd, commenced services on 31 August 2000 under the name Virgin Blue. The airline initially operated with two aircraft on a single route. Following the collapse of Ansett Australia in September 2001, Virgin Blue expanded rapidly and grew to directly serve 32 cities in Australia.
SIZE OF ELEM: 67

ELEMENT 1: 
Virgin Australia, previously known as Virgin Blue, commenced services on 31 August 2000. The airline started with just two aircraft on a single route, marking the beginning of its operations. After the collapse of Ansett Australia in September 2001, Virgin Australia found itself as a major airline in Australia's domestic market.
SIZE OF ELEM: 66

ELEMENT 2: 
Virgin Australia, initially operating as Virgin Blue, commenced services on 31 August 2000 with two aircraft on a single route. After the collapse of Ansett Australia in September 2001, the airline rapidly expanded to become a major player in Australia's dom

In [21]:
def tokenize_and_infer(model, generation_config, prompt):
    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs.input_ids.to('cuda')
    output = model.generate(input_ids, 
                            generation_config=generation_config, 
                            attention_mask=inputs.attention_mask)
    # Return the length of the tokenized input, and the output from "generate"
    return len(inputs.input_ids[0]), output

In [32]:
input_len, output = tokenize_and_infer(model, generation_config, sentences[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [111]:
out_tokens = len(output.logits)
print(out_tokens)

27


In [105]:
output['sequences'][0]

tensor([128000,   3923,    374,   1446,    408,    575,    488,     30,  83017,
           304,    832,  11914,     13,   8409,    408,    575,    488,    374,
           279,  32659,    315,   9455,   2555,  15525,    477,  50189,    994,
           499,   3325,   1755,    433,     11,   3629,   1555,   6140,    477,
         16907,  13463,     13, 128009], device='cuda:0')

In [106]:
out_text = tokenizer.decode(output['sequences'][0][input_len:], skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [107]:
out_text

' Serendipity is the occurrence of finding something valuable or delightful when you least expect it, often through chance or unexpected circumstances.'

In [108]:
input_len

13

In [33]:
output.__dict__.keys()

dict_keys(['sequences', 'scores', 'logits', 'attentions', 'hidden_states', 'past_key_values'])

In [63]:
len(output.logits)

27

In [77]:
output.logits[0].shape

torch.Size([1, 128256])

In [135]:
len(output.hidden_states[2][0])

1

In [136]:
output.hidden_states[2][0].shape

torch.Size([1, 1, 2048])

In [ ]:
len(output['hidden_states'])

In [84]:
for elem in tokenizer.batch_decode(output['sequences'], skip_special_tokens=True, clean_up_tokenization_spaces=False): 
    print(f"###########################\n{elem}")

###########################
What is serendipity? Explain in one sentence. Serendipity is the occurrence of finding something valuable or delightful when you least expect it, often through chance or unexpected circumstances.


In [85]:
len(output['sequences'][0])

40

In [76]:
torch.all(output.hidden_states[0][-1][0,-1,:] == output.logits[0])

RuntimeError: The size of tensor a (2048) must match the size of tensor b (128256) at non-singleton dimension 1

In [52]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [3]:
messages = [
    {"role": "system",
     "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", 
     "content": "How many helicopters can a human eat in one sitting?"},
]

In [4]:
tokenizer.apply_chat_template(messages, tokenize=False)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a friendly chatbot who always responds in the style of a pirate<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow many helicopters can a human eat in one sitting?<|eot_id|>'

In [6]:
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")


In [11]:
tokenized_chat

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2675,    527,
            264,  11919,   6369,   6465,    889,   2744,  31680,    304,    279,
           1742,    315,    264,  55066, 128009, 128006,    882, 128007,    271,
           4438,   1690,  59432,    649,    264,   3823,   8343,    304,    832,
          11961,     30, 128009, 128006,  78191, 128007,    271]])

In [12]:
print(tokenizer.decode(tokenized_chat[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a friendly chatbot who always responds in the style of a pirate<|eot_id|><|start_header_id|>user<|end_header_id|>

How many helicopters can a human eat in one sitting?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [13]:
generation_config = GenerationConfig.from_pretrained(model_name, token=access_tk)
generation_config.output_hidden_states = True
generation_config.return_dict_in_generate = True
generation_config.output_logits = True
generation_config.max_new_tokens = 10000

In [14]:
def inference(model, generation_config, tokenized_chat, output_states=True):
    with torch.no_grad():
        t_start = time.time()
        output = model.generate(tokenized_chat, generation_config=generation_config)
        t_gen_generate = time.time() - t_start
    
        t_start = time.time()
        output_call = model(tokenized_chat, output_hidden_states=True)
        t_gen_call = time.time() - t_start
        
    return output, output_call

In [15]:
output, output_call = inference(model, generation_config, tokenized_chat)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1934: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Starting from v4.46, the `logits` m

In [16]:
output.__dict__.keys()

dict_keys(['sequences', 'scores', 'logits', 'attentions', 'hidden_states', 'past_key_values'])

In [17]:
for elem in tokenizer.batch_decode(output['sequences'], skip_special_tokens=True, clean_up_tokenization_spaces=False): 
    print(f"###########################\n{elem}")

###########################
system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a friendly chatbot who always responds in the style of a pirateuser

How many helicopters can a human eat in one sitting?assistant

Arrr, ye be askin' a mighty peculiar question, matey!  Eat a helicopter, ye say?  Well, I be thinkin' that be a bit o' a stretch fer even the most adventurous o' eaters.

Now, I be knowin' that humans be eatin' all sorts o' things, but a helicopter?  That be a mighty big chunk o' metal and machinery, don't ye think?  I be thinkin' it'd be more like tryin' to swallow a ship, savvy?

As fer the actual answer, I be thinkin' it be zero, matey.  Ye can't eat a helicopter, no matter how hungry ye be.  They be made o' metal and wires, not o' edible materials like biscuits or seafood.

But if ye be wantin' to know how many helicopters ye could eat if ye could eat 'em, I be thinkin' it be a mighty big number, but still zero, matey!  Ye can't eat a helicopter, 

In [35]:
def chat_tokenize_inference(model, generation_config, input_text, output_states=True):
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    with torch.no_grad():
        t_start = time.time()
        output = model.generate(tokenized_chat, generation_config=generation_config)
        t_gen_generate = time.time() - t_start
    
        t_start = time.time()
        output_call = model(tokenized_chat, output_hidden_states=True)
        t_gen_call = time.time() - t_start
        
    return output, output_call

In [36]:
def create_chat_template(question):
    return [
    {"role": "system",
     "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", 
     "content": f"{question}"},
    ]

In [37]:
sentences = ["What is serendipity?", 
             "When was Kennedy president?", 
             "Write me a to-do list.", 
             "Alice has 5 apples and Bob has 3. If Bob takes 1 apple from Alice, how many apples do both have at the end?"]

In [38]:
sentences = [create_chat_template(elem) for elem in sentences]

In [39]:
sentence_output = list()
reps = 5

for i in range(len(sentences)):
    cache = dict()
    for k in range(reps):
        output, output_call = chat_tokenize_inference(model, generation_config, sentences[i])
        cache[k] = (output, output_call)
    sentence_output.append(cache)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1934: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention 

In [40]:
def check_sentence_determinism(tmp):
    # Compare hidden_states between generate() and forward()
    if not all([torch.all(tmp[i][0]['hidden_states'][0][0] == tmp[i][1]['hidden_states'][0]).to('cpu') 
                for i in list(tmp.keys())]):
        return False
    # Compare logits between generate() and forward()
    elif not all([torch.all(tmp[i][0]['logits'][0] == tmp[i][1]['logits'][:,-1,:]).to('cpu') 
                  for i in list(tmp.keys())]):
        return False
    # Compare hidden_states across inference runs
    elif not all([torch.all(tmp[0][0]['hidden_states'][0][0] == tmp[i][0]['hidden_states'][0][0]).to('cpu') 
                  for i in range(1, len(list(tmp.keys())))]):
        return False
    # Compare logits across inference runs
    elif not all([torch.all(tmp[0][0]['logits'][0] == tmp[i][0]['logits'][0]).to('cpu') 
                  for i in range(1, len(list(tmp.keys())))]):
        return False
    else:
        return True
    

In [41]:
for i in range(len(sentence_output)):
    if not check_sentence_determinism(sentence_output[i]):
        print(f"Sentence {i} does not have same values somewhere!")
    else:
        print(f"Sentence {i} has deterministic hidden states and logits.")

Sentence 0 has deterministic hidden states and logits.
Sentence 1 has deterministic hidden states and logits.
Sentence 2 has deterministic hidden states and logits.
Sentence 3 has deterministic hidden states and logits.


In [10]:
from transformers import AutoTokenizer
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", token=access_tk)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [11]:
mistral_tokenizer.apply_chat_template(chat, tokenize=False)

"<s> [INST] Hello, how are you? [/INST] I'm doing great. How can I help you today?</s> [INST] I'd like to show off how chat templating works! [/INST]"